In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
import math
from pyts.image import RecurrencePlot, GramianAngularField
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import load_model, Sequential
import json
import time
from collections import deque
import warnings
from pickle import dump, load

file_path = '/Users/mymac/Google Drive/My Drive/Forex_Robot/'

In [2]:
def apply_impact(val, impact):
    return float(val) * float(impact)

def clean_value(val):
    return float(val)
    # new_val = val.replace(',', '')

    # if 'M' in new_val:
    #     num, _ = new_val.split('M')

    #     return float(num) * 1000000

    # elif 'B' in new_val:
    #     num, _ = new_val.split('B')

    #     return float(num) * 1000000000

    # elif 'k' in new_val:
    #     num, _ = new_val.split('k')

    #     return float(num) * 1000

    # elif '%' in new_val:
    #     num, _ = new_val.split('%')

    #     return float(num) / 100

    # else:
    #     return float(new_val)

def forecast_ratio(forecast, actual):
    if str(forecast) == 'nan' or str(actual) == 'nan':
        return 0.0

    forecast_clean = clean_value(forecast)
    actual_clean = clean_value(actual)
    
    if actual_clean == 0:
        return 0

    return forecast_clean / actual_clean

def previous_ratio(previous, actual):
    if str(previous) == 'nan' or str(actual) == 'nan':
        return 0.0

    previous_clean = clean_value(previous)
    actual_clean = clean_value(actual)
    
    if actual_clean == 0:
        return 0

    return previous_clean / actual_clean


# news = pd.read_csv(file_path + 'events.csv')
# news.Date = pd.to_datetime(news.Date, utc=True)
# news = news.loc[(news['Currency_Code'] == 'AUD') | (news['Currency_Code'] == 'USD')]
# news.drop(news[(news['Impact'] != 'low') & (news['Impact'] != 'med') & (news['Impact'] != 'high')].index, inplace=True)
# news.loc[news['Impact'] == 'low', 'Impact'] = 1
# news.loc[news['Impact'] == 'med', 'Impact'] = 2
# news.loc[news['Impact'] == 'high', 'Impact'] = 3
# news['Forecast_Ratio'] = news.apply(lambda row: forecast_ratio(row['Forecast'], row['Actual']), axis=1)
# news['Previous_Ratio'] = news.apply(lambda row: previous_ratio(row['Previous'], row['Actual']), axis=1)
# news.drop(['Currency_Code', 'Actual', 'Forecast', 'Previous'], axis=1, inplace=True)
# by_date = news.groupby('Date')
# impact, all_day, forecast_r, previous_r = by_date['Impact'].mean().reset_index(), by_date['All_Day'].mean().reset_index(), by_date['Forecast_Ratio'].mean().reset_index(), by_date['Previous_Ratio'].mean().reset_index()
# news = news.iloc[0:0]
# news['Date'], news['Impact'], news['All_Day'], news['Forecast_Ratio'], news['Previous_Ratio'] = impact['Date'], impact['Impact'], all_day['All_Day'], forecast_r['Forecast_Ratio'], previous_r['Previous_Ratio']
# news.tail()

In [3]:
# currencies = ['Usd_Chf', 'Gbp_Usd', 'Usd_Cad', 'Aud_Usd', 'Eur_Usd', 'Usd_Jpy', 'Nzd_Usd']
# GBP_JPY, CAD_JPY, EUR_JPY, GBP_CAD, EUR_GBP
# 'Gbp_Usd', 'Eur_Usd', 'Usd_Cad', 'Usd_Jpy', 'Aud_Usd'
currencies = ['Eur_Gbp']
years = '2012-2022'

dfs, df_longs = [], []

# news = pd.read_csv(file_path + 'events_2016-2022.csv')
# news = news.rename(columns={'Start': 'Date'})
# news.Date = pd.to_datetime(news.Date)

for currency_pair in currencies:
    currency1, currency2 = currency_pair.split('_')
    currency1, currency2 = currency1.upper(), currency2.upper()

    news = pd.read_csv(file_path + 'events.csv')
    news.Date = pd.to_datetime(news.Date, utc=True)
    news = news.loc[(news['Currency_Code'] == currency1) | (news['Currency_Code'] == currency2)]
    news.drop(news[(news['Impact'] != 'low') & (news['Impact'] != 'med') & (news['Impact'] != 'high')].index, inplace=True)
    news.loc[news['Impact'] == 'low', 'Impact'] = 1
    news.loc[news['Impact'] == 'med', 'Impact'] = 2
    news.loc[news['Impact'] == 'high', 'Impact'] = 3
    news['Actual_Class'] = news.apply(lambda row: clean_value(row['Actual']), axis=1)
    news['Previous_Class'] = news.apply(lambda row: clean_value(row['Previous']), axis=1)
    # news['Actual_Class'] = news.apply(lambda row: apply_impact(row['Actual'], row['Impact']), axis=1)
    # news['Previous_Class'] = news.apply(lambda row: apply_impact(row['Actual'], row['Impact']), axis=1)
    news.drop(['Currency_Code', 'Actual', 'Previous'], axis=1, inplace=True)
    by_date = news.groupby('Date')
    impact, actual, previous = by_date['Impact'].mean().reset_index(), by_date['Actual_Class'].mean().reset_index(), by_date['Previous_Class'].mean().reset_index()
    news = news.iloc[0:0]
    news['Date'], news['Impact'], news['Actual_Class'], news['Previous_Class'] = impact['Date'], impact['Impact'], actual['Actual_Class'], previous['Previous_Class']

    df = pd.read_csv(file_path + f'Oanda_{currency_pair}_M5_{years}.csv')
    df.Date = pd.to_datetime(df.Date, utc=True)
    df.reset_index(drop=True, inplace=True)
    df = pd.merge(df, news, how='left', on='Date')
    df.reset_index(drop=True, inplace=True)
    df = df.fillna(method='ffill')
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)

    dfs.append(df)

    df_long = pd.read_csv(file_path + f'Oanda_{currency_pair}_M30_{years}.csv')
    df_long.Date = pd.to_datetime(df_long.Date, utc=True)
    df_long.dropna(inplace=True)
    df_long.reset_index(drop=True, inplace=True)
    df_longs.append(df_long)

<ipython-input-3-45153325c917>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news['Date'], news['Impact'], news['Actual_Class'], news['Previous_Class'] = impact['Date'], impact['Impact'], actual['Actual_Class'], previous['Previous_Class']


In [4]:
news.dtypes

Date              datetime64[ns, UTC]
Impact                        float64
Actual_Class                  float64
Previous_Class                float64
dtype: object

In [5]:
def add_fractal(df, i, look_back=3):
    if i >= look_back and i < df.shape[0] - look_back:
        lows = []
        highs = []

        for j in range(1, look_back + 1):
            prev_mid_low, prev_mid_high = df.loc[df.index[i - j], ['Mid_Low', 'Mid_High']]
            future_mid_low, future_mid_high = df.loc[df.index[i + j], ['Mid_Low', 'Mid_High']]

            lows.append(float(prev_mid_low))
            lows.append(float(future_mid_low))
            highs.append(float(prev_mid_high))
            highs.append(float(future_mid_high))

        mid_low, mid_high = df.loc[df.index[i], ['Mid_Low', 'Mid_High']]

        if float(mid_low) < min(lows):
            return float(mid_low), 1.0

        elif float(mid_high) > max(highs):
            return float(mid_high), 0.0

        else:
            return np.nan, np.nan

    else:
        return np.nan, np.nan

def psar(barsdata, iaf=0.02, maxaf=0.2):
    length = len(barsdata)
    high = list(barsdata['Mid_High'])
    low = list(barsdata['Mid_Low'])
    close = list(barsdata['Mid_Close'])
    psar = close[0:len(close)]
    bull = True
    af = iaf
    hp = high[0]
    lp = low[0]
    for i in range(2, length):
        if bull:
            psar[i] = psar[i - 1] + af * (hp - psar[i - 1])
        else:
            psar[i] = psar[i - 1] + af * (lp - psar[i - 1])
        reverse = False
        if bull:
            if low[i] < psar[i]:
                bull = False
                reverse = True
                psar[i] = hp
                lp = low[i]
                af = iaf
        else:
            if high[i] > psar[i]:
                bull = True
                reverse = True
                psar[i] = lp
                hp = high[i]
                af = iaf
        if not reverse:
            if bull:
                if high[i] > hp:
                    hp = high[i]
                    af = min(af + iaf, maxaf)
                if low[i - 1] < psar[i]:
                    psar[i] = low[i - 1]
                if low[i - 2] < psar[i]:
                    psar[i] = low[i - 2]
            else:
                if low[i] < lp:
                    lp = low[i]
                    af = min(af + iaf, maxaf)
                if high[i - 1] > psar[i]:
                    psar[i] = high[i - 1]
                if high[i - 2] > psar[i]:
                    psar[i] = high[i - 2]
    return psar


def atr(high, low, close, lookback=14):
    high_low = high - low
    high_close = np.abs(high - close.shift())
    low_close = np.abs(low - close.shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)

    return true_range.rolling(lookback).sum() / lookback


def rsi(closes, periods=14):
    close_delta = closes.diff()

    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))

    return rsi

  
def adx(high, low, close, lookback=14):
    plus_dm = high.diff()
    minus_dm = low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0
    
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.rolling(lookback).mean()
    
    plus_di = 100 * (plus_dm.ewm(alpha = 1/lookback).mean() / atr)
    minus_di = abs(100 * (minus_dm.ewm(alpha = 1/lookback).mean() / atr))
    dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
    adx = ((dx.shift(1) * (lookback - 1)) + dx) / lookback
    adx_smooth = adx.ewm(alpha = 1/lookback).mean()

    return adx_smooth


def stoch(high, low, close, lookback=14):
    high_lookback = high.rolling(lookback).max()
    low_lookback = low.rolling(lookback).min()
    slow_k = (close - low_lookback) * 100 / (high_lookback - low_lookback)
    slow_d = slow_k.rolling(3).mean()

    return slow_k, slow_d

def stoch_rsi(data, k_window=3, d_window=3, window=14):
    min_val = data.rolling(window=window, center=False).min()
    max_val = data.rolling(window=window, center=False).max()

    stoch = ((data - min_val) / (max_val - min_val)) * 100

    slow_k = stoch.rolling(window=k_window, center=False).mean()

    slow_d = slow_k.rolling(window=d_window, center=False).mean()

    return slow_k, slow_d

def n_macd(macd, macdsignal, lookback=50):
    n_macd = 2 * (((macd - macd.rolling(lookback).min()) / (macd.rolling(lookback).max() - macd.rolling(lookback).min()))) - 1
    n_macdsignal = 2 * (((macdsignal - macdsignal.rolling(lookback).min()) / (macdsignal.rolling(lookback).max() - macdsignal.rolling(lookback).min()))) - 1

    return n_macd, n_macdsignal

def chop(df, lookback=14):
    atr1 = atr(df, lookback=1)
    high, low = df['Mid_High'], df['Mid_Low']

    chop = np.log10(atr1.rolling(lookback).sum() / (high.rolling(lookback).max() - low.rolling(lookback).min())) / np.log10(lookback)

    return chop

def vo(volume, short_lookback=5, long_lookback=10):
    short_ema =  pd.Series.ewm(volume, span=short_lookback).mean()
    long_ema = pd.Series.ewm(volume, span=long_lookback).mean()

    volume_oscillator = (short_ema - long_ema) / long_ema

    return volume_oscillator

def bar_lengths(bar_lens, window=36):
    return bar_lens.rolling(window=window).mean(), bar_lens.rolling(window=window).std()

def sar_lengths(opens, sars, window=36):
    diffs = abs(opens - sars.shift(1))

    return diffs.rolling(window=window).mean(), diffs.rolling(window=window).std()

def supertrend(barsdata, atr_len=3, mult=3):
    curr_atr = atr(barsdata['Mid_High'], barsdata['Mid_Low'], barsdata['Mid_Close'], lookback=atr_len)
    highs, lows = barsdata['Mid_High'], barsdata['Mid_Low']
    hl2 = (highs + lows) / 2
    final_upperband = upper_band = hl2 + mult * curr_atr
    final_lowerband = lower_band = hl2 - mult * curr_atr

    # initialize Supertrend column to True
    supertrend = [True] * len(df)

    close = barsdata['Mid_Close']
    
    for i in range(1, len(df.index)):
        curr, prev = i, i - 1
        
        # if current close price crosses above upperband
        if close[curr] > final_upperband[prev]:
            supertrend[curr] = True

        # if current close price crosses below lowerband
        elif close[curr] < final_lowerband[prev]:
            supertrend[curr] = False

        # else, the trend continues
        else:
            supertrend[curr] = supertrend[prev]
            
            # adjustment to the final bands
            if supertrend[curr] == True and final_lowerband[curr] < final_lowerband[prev]:
                final_lowerband[curr] = final_lowerband[prev]

            if supertrend[curr] == False and final_upperband[curr] > final_upperband[prev]:
                final_upperband[curr] = final_upperband[prev]

    return supertrend, final_upperband, final_lowerband

def heikin_ashi(opens, highs, lows, closes):
    ha_close = list((opens + highs + lows + closes) / 4)
    ha_opens = []

    opens_list, closes_list = list(opens), list(closes)

    for i in range(len(ha_close)):
        if i == 0:
            ha_opens.append((opens_list[i] + closes_list[i]) / 2)

        else:
            ha_opens.append((ha_opens[i - 1] + ha_close[i - 1]) / 2)

    ha_highs = list(pd.DataFrame({'ha_open': ha_opens, 'ha_close': ha_close, 'high': list(highs)}).max(axis=1))
    ha_lows = list(pd.DataFrame({'ha_open': ha_opens, 'ha_close': ha_close, 'low': list(lows)}).min(axis=1))

    return ha_opens, ha_highs, ha_lows, ha_close

def trend_indicator(opens, highs, lows, closes, ema_period=50, smoothing_period=10):
    ha_open, ha_high, ha_low, ha_close = heikin_ashi(opens, highs, lows, closes)

    ha_o_ema = pd.Series.ewm(pd.DataFrame({'ha_open': ha_open}), span=ema_period).mean()
    ha_h_ema = pd.Series.ewm(pd.DataFrame({'ha_high': ha_high}), span=ema_period).mean()
    ha_l_ema = pd.Series.ewm(pd.DataFrame({'ha_low': ha_low}), span=ema_period).mean()
    ha_c_ema = pd.Series.ewm(pd.DataFrame({'ha_close': ha_close}), span=ema_period).mean()

    return pd.Series.ewm(ha_o_ema, span=smoothing_period).mean(), pd.Series.ewm(ha_h_ema, span=smoothing_period).mean(), pd.Series.ewm(ha_l_ema, span=smoothing_period).mean(), pd.Series.ewm(ha_c_ema, span=smoothing_period).mean()

def qqe_mod(barsdata, rsi_period=6, smoothing=5, qqe_factor=5, qqe2_factor=1.61, threshold=3, mult=0.35, sma_length=50):
    wilders_period = rsi_period * 2 - 1

    curr_rsi = rsi(barsdata, periods=rsi_period)
    rsi_ema = pd.Series.ewm(curr_rsi, span=smoothing).mean()
    atr_rsi = abs(rsi_ema.shift(1) - rsi_ema)
    atr_rsi_ema = pd.Series.ewm(atr_rsi, span=wilders_period).mean()
    dar = pd.Series.ewm(atr_rsi_ema, span=wilders_period).mean() * qqe_factor

    newshortband = rsi_ema + dar
    newlongband = rsi_ema - dar

    rsi_ema_list = list(rsi_ema)

    longband = [0]
    for i in range(1, len(rsi_ema_list)):
        if rsi_ema_list[i - 1] > longband[i - 1] and rsi_ema_list[i] > longband[i - 1]:
            longband.append(max(longband[i - 1],newlongband[i]))

        else:
            longband.append(newlongband[i])

    shortband = [0]
    for i in range(1,len(rsi_ema_list)):
        if rsi_ema_list[i - 1] < shortband[i - 1] and rsi_ema_list[i] < shortband[i - 1]:
            shortband.append(min(shortband[i - 1],newshortband[i]))
            
        else:
            shortband.append(newshortband[i])

    longband = pd.Series(longband)
    shortband = pd.Series(shortband)

    trend = np.where(rsi_ema > longband.shift(1), 1, -1)    
    fastatrrsitl = pd.Series(np.where(trend == 1, longband, shortband))

    basis = (fastatrrsitl - 50).rolling(window=sma_length).mean()
    dev = (fastatrrsitl - 50).rolling(window=sma_length).std() * mult
    upper = basis + dev
    lower = basis - dev

    greenbar1 = rsi_ema - 50 > threshold
    greenbar2 = rsi_ema - 50 > upper
    redbar1 = rsi_ema - 50 < threshold
    redbar2 = rsi_ema - 50 < lower

    # uptrend = np.where((greenbar1 & greenbar2), True, False)
    # downtrend = np.where((redbar1 & redbar2), True, False)

    uptrend = np.where((greenbar2), True, False)
    downtrend = np.where((redbar2), True, False)

    return uptrend, downtrend

def williams_r(highs, lows, closes, length=21, ema_length=15):
    highest_highs = highs.rolling(window=length).max()
    lowest_lows = lows.rolling(window=length).min()

    willy = 100 * (closes - highest_highs) / (highest_highs - lowest_lows)
    willy_ema = pd.Series.ewm(willy, span=ema_length).mean()

    return willy, willy_ema

def squeeze(barsdata, length=20, length_kc=20, mult=1.5):
    # Bollinger bands
    m_avg = barsdata['Mid_Close'].rolling(window=length).mean()
    m_std = barsdata['Mid_Close'].rolling(window=length).std(ddof=0)
    upper_bb = m_avg + mult * m_std
    lower_bb = m_avg - mult * m_std

    # Keltner channel
    tr0 = abs(barsdata['Mid_High'] - barsdata['Mid_Low'])
    tr1 = abs(barsdata['Mid_High'] - barsdata['Mid_Close'].shift())
    tr2 = abs(barsdata['Mid_Low'] - barsdata['Mid_Close'].shift())
    tr = pd.concat([tr0, tr1, tr2], axis=1).max(axis=1)
    range_ma = tr.rolling(window=length_kc).mean()
    upper_kc = m_avg + range_ma * mult
    lower_kc = m_avg - range_ma * mult

    # Squeeze
    squeeze_on = (lower_bb > lower_kc) & (upper_bb < upper_kc)

    return squeeze_on

In [6]:
bid_ask_mid_prices_list = []

for df in dfs:
    # Add technical indicators (for additional features)
    df['ema200'] = pd.Series.ewm(df['Mid_Close'], span=200).mean()
    df['ema100'] = pd.Series.ewm(df['Mid_Close'], span=100).mean()

    df['rsi'] = rsi(df['Mid_Close'])
    df['rsi_sma'] = df['rsi'].rolling(50).mean()
    df['adx'] = adx(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
    df['macd'] = pd.Series.ewm(df['Mid_Close'], span=12).mean() - pd.Series.ewm(df['Mid_Close'], span=26).mean()
    df['macdsignal'] = pd.Series.ewm(df['macd'], span=9).mean()
    df['slowk_rsi'], df['slowd_rsi'] = stoch_rsi(df['rsi'])

    df['squeeze_on'] = squeeze(df)

    # tups = [add_fractal(df, i) for i in range(df.shape[0])]
    # key_levels, is_supports = [tup[0] for tup in tups], [tup[1] for tup in tups]
    # df['key_level'], df['is_support'] = key_levels, is_supports
    # df = df.fillna(method='ffill')

    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)

    # Extract the bid and ask prices and fractals and remove them from the df
    bid_ask_mid_prices = df[['Date','Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Ask_Close', 'Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close']]
    df.drop(['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Ask_Close', 'Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close', 'Volume'], axis=1, inplace=True)
    bid_ask_mid_prices_list.append(bid_ask_mid_prices)

    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)

for df_long in df_longs:
    df_long['ema200'] = pd.Series.ewm(df_long['Mid_Close'], span=200).mean()
    df_long['ema100'] = pd.Series.ewm(df_long['Mid_Close'], span=100).mean()

    df_long['rsi'] = rsi(df_long['Mid_Close'])
    df_long['rsi_sma'] = df_long['rsi'].rolling(50).mean()
    df_long['adx'] = adx(df_long['Mid_High'], df_long['Mid_Low'], df_long['Mid_Close'])
    df_long['macd'] = pd.Series.ewm(df_long['Mid_Close'], span=12).mean() - pd.Series.ewm(df_long['Mid_Close'], span=26).mean()
    df_long['macdsignal'] = pd.Series.ewm(df_long['macd'], span=9).mean()
    df_long['slowk_rsi'], df_long['slowd_rsi'] = stoch_rsi(df_long['rsi'])

    df_long['squeeze_on'] = squeeze(df_long)

    # tups = [add_fractal(df_long, i) for i in range(df_long.shape[0])]
    # key_levels, is_supports = [tup[0] for tup in tups], [tup[1] for tup in tups]
    # df_long['key_level'], df_long['is_support'] = key_levels, is_supports
    # df_long = df_long.fillna(method='ffill')

    df_long.dropna(inplace=True)
    df_long.reset_index(drop=True, inplace=True)

    df_long.drop(['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Ask_Close', 'Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close', 'Volume'], axis=1, inplace=True)

    df_long.dropna(inplace=True)
    df_long.reset_index(drop=True, inplace=True)

In [7]:
for i in range(len(dfs)):
    df, df_long = dfs[i], df_longs[i]

    df = pd.merge(df, df_long, how='left', on='Date')
    df.reset_index(drop=True, inplace=True)
    df = df.fillna(method='ffill')
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)

    dfs[i] = df

In [8]:
# for df in dfs:
#     df.drop(['Bid_Open_x', 'Bid_High_x', 'Bid_Low_x', 'Bid_Close_x', 'Ask_Open_x', 'Ask_High_x', 'Ask_Low_x', 'Ask_Close_x', 'Mid_Open_x', 'Mid_High_x', 'Mid_Low_x', 'Mid_Close_x', 'Volume_x'], axis=1, inplace=True)
#     df.drop(['Bid_Open_y', 'Bid_High_y', 'Bid_Low_y', 'Bid_Close_y', 'Ask_Open_y', 'Ask_High_y', 'Ask_Low_y', 'Ask_Close_y', 'Mid_Open_y', 'Mid_High_y', 'Mid_Low_y', 'Mid_Close_y', 'Volume_y'], axis=1, inplace=True)

In [9]:
dfs[0].head()

,Date,Impact,Actual_Class,Previous_Class,ema200_x,ema100_x,rsi_x,rsi_sma_x,adx_x,macd_x,...,ema200_y,ema100_y,rsi_y,rsi_sma_y,adx_y,macd_y,macdsignal_y,slowk_rsi_y,slowd_rsi_y,squeeze_on_y
0,2018-01-02 14:30:00+00:00,2.0,-1.0,0.0,0.889744,0.889698,41.295294,48.733898,27.707496,-0.000221,...,0.887962,0.888581,42.814245,56.104998,12.831831,0.000065,0.000249,2.415608,7.051329,True
1,2018-01-02 14:35:00+00:00,2.0,-1.0,0.0,0.889725,0.889674,36.121716,47.882063,27.648434,-0.000236,...,0.887962,0.888581,42.814245,56.104998,12.831831,0.000065,0.000249,2.415608,7.051329,True
2,2018-01-02 14:40:00+00:00,2.0,-1.0,0.0,0.889709,0.889654,39.337101,47.064948,27.693058,-0.000236,...,0.887962,0.888581,42.814245,56.104998,12.831831,0.000065,0.000249,2.415608,7.051329,True
3,2018-01-02 14:45:00+00:00,2.0,-1.0,0.0,0.889689,0.889629,35.807869,46.238375,27.761674,-0.000249,...,0.887962,0.888581,42.814245,56.104998,12.831831,0.000065,0.000249,2.415608,7.051329,True
4,2018-01-02 14:50:00+00:00,2.0,-1.0,0.0,0.889667,0.889600,32.652972,45.282686,28.213402,-0.000272,...,0.887962,0.888581,42.814245,56.104998,12.831831,0.000065,0.000249,2.415608,7.051329,True


In [10]:
for i in range(len(dfs)):
    df, bid_ask_mid_prices = dfs[i], bid_ask_mid_prices_list[i]

    stard_idx = len(bid_ask_mid_prices) - len(df)

    bid_ask_mid_prices = bid_ask_mid_prices.iloc[stard_idx:, :]
    bid_ask_mid_prices.dropna(inplace=True)
    bid_ask_mid_prices.reset_index(drop=True, inplace=True)

    assert len(bid_ask_mid_prices) == len(df)

    bid_ask_mid_prices_list[i] = bid_ask_mid_prices

/Users/mymac/miniforge3/envs/forex/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [11]:
look_back_size = 150

In [12]:
def grab_image_data(subset):
  # rp_transformer = RecurrencePlot()
  # rp_subset = rp_transformer.transform(subset)

  # return rp_subset

  gasf_transformer = GramianAngularField(method='summation')
  gasf_subset = gasf_transformer.transform(subset)

  return gasf_subset

  # gadf_transformer = GramianAngularField(method='difference')
  # gadf_subset = gadf_transformer.transform(subset)

  # return gadf_subset

  # image_data = np.append(rp_subset, gasf_subset, axis=-1)
  # image_data = np.append(image_data, gadf_subset ,axis=-1)
  
  # return image_data

In [13]:
# buys_list = []
# sells_list = []
# nones_list = []

# value_per_pip = 1.0
# amounts_per_day = [-0.00008, -0.0001, -0.00012]
# spread_cutoff = 0.10
# risk_reward_ratio = 1.5

# def get_n_units(trade_type, stop_loss, ask_open, bid_open, mid_open, currency_pair):
#     _, second = currency_pair.split('_')
  
#     pips_to_risk = ask_open - stop_loss if trade_type == 'buy' else stop_loss - bid_open
#     pips_to_risk_calc = pips_to_risk * 10000 if second != 'Jpy' else pips_to_risk * 100

#     if second == 'Usd':
#         per_pip = 0.0001

#     else:
#         per_pip = 0.0001 / mid_open if second != 'Jpy' else 0.01 / mid_open

#     n_units = int(50 / (pips_to_risk_calc * per_pip))

#     return n_units

# def calculate_day_fees(start_date, end_date, n_units):
#     curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
#     num_days = np.busday_count(start_date.date(), end_date.date())

#     return num_days * curr_fee

# for idx in range(len(currencies)):
#     currency_pair, df, df_long, bid_ask_mid_prices = currencies[idx], dfs[idx], df_longs[idx], bid_ask_mid_prices_list[idx]
#     rounding = 3 if 'Jpy' in currency_pair else 5
#     buys, sells, nones = [], [], []
#     trade, prev_year = None, None

#     print(f'RUNNING SIMULATION FOR {currency_pair}...')

#     for i in range(look_back_size, len(df)):
#         curr_date = df.loc[df.index[i], 'Date']
#         if prev_year is None or curr_date.year > prev_year:
#             prev_year = curr_date.year
#             print(prev_year)
#             print(f'Buys: {len(buys)}')
#             print(f'Sells: {len(sells)}')
#             print(f'Nones: {len(nones)}\n')
#         curr_long = df_long.loc[df_long['Date'] < curr_date]
#         if len(curr_long) < look_back_size + 1:
#             continue
#         ema200_2, ema100_2, atr2, rsi2, rsi_sma2, vo2, macd2, macdsignal2 = df.loc[df.index[i - 2], ['ema200', 'ema100', 'atr', 'rsi', 'rsi_sma', 'vo', 'macd', 'macdsignal']]
#         ema200_1, ema100_1, atr1, rsi1, rsi_sma1, vo1, adx1, atr_sma1, macd1, macdsignal1 = df.loc[df.index[i - 1], ['ema200', 'ema100', 'atr', 'rsi', 'rsi_sma', 'vo', 'adx', 'atr_sma', 'macd', 'macdsignal']]
#         mid_open2, mid_close2, mid_low2, mid_high2 = bid_ask_mid_prices.loc[bid_ask_mid_prices.index[i - 2], ['Mid_Open', 'Mid_Close', 'Mid_Low', 'Mid_High']]
#         mid_open1, mid_close1, mid_low1, mid_high1 = bid_ask_mid_prices.loc[bid_ask_mid_prices.index[i - 1], ['Mid_Open', 'Mid_Close', 'Mid_Low', 'Mid_High']]
#         curr_ao, curr_bo, curr_mid_open, curr_ask_low, curr_bid_high = bid_ask_mid_prices.loc[bid_ask_mid_prices.index[i], ['Ask_Open', 'Bid_Open', 'Mid_Open', 'Ask_Low', 'Bid_High']]
#         spread = abs(curr_ao - curr_bo)

#         emas_buy_signal = ema200_1 < ema100_1
#         emas_sell_signal = ema200_1 > ema100_1

#         # sideways = adx1 < 20 or atr1 < atr_sma1
#         sideways = False

#         rsi_buy_signal = rsi1 > rsi_sma1
#         rsi_sell_signal = rsi1 < rsi_sma1

#         macd_vals = [0, macd2, macdsignal2, macd1, macdsignal1]

#         macd_buy_signal = macd2 < macdsignal2 and macd1 > macdsignal1 and max(macd_vals) == 0
#         macd_sell_signal = macd2 > macdsignal2 and macd1 < macdsignal1 and min(macd_vals) == 0

#         if trade is None:
#             if macd_buy_signal and emas_buy_signal and rsi_buy_signal and not sideways:
#                 lowest_low = min(list(bid_ask_mid_prices.loc[bid_ask_mid_prices.index[i - 12:i], 'Mid_Low']))
                
#                 open_price = float(curr_ao)
#                 pullback = float(lowest_low) - spread

#                 stop_loss = round(pullback, rounding)

#                 if stop_loss < open_price:
#                     curr_pips_to_risk = open_price - stop_loss

#                     if spread <= curr_pips_to_risk * spread_cutoff:
#                         stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), rounding)

#                         n_units = get_n_units('buy', stop_loss, curr_ao, curr_bo, curr_mid_open, currency_pair)

#                         trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
#                                                         'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
#                                                         'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

#             elif macd_sell_signal and emas_sell_signal and rsi_sell_signal and not sideways:
#                 highest_high = max(list(bid_ask_mid_prices.loc[bid_ask_mid_prices.index[i - 12:i], 'Mid_High']))

#                 open_price = float(curr_bo)
#                 pullback = float(highest_high) + spread

#                 stop_loss = round(pullback, rounding)

#                 if stop_loss > open_price:
#                     curr_pips_to_risk = stop_loss - open_price

#                     if spread <= curr_pips_to_risk * spread_cutoff:
#                         stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), rounding)

#                         n_units = get_n_units('sell', stop_loss, curr_ao, curr_bo, curr_mid_open, currency_pair)

#                         trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
#                                 'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
#                                 'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

#         if trade is not None:
#             for j in range(i, len(df)):
#                 curr_date = df.loc[df.index[j], 'Date']
#                 curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = bid_ask_mid_prices.loc[bid_ask_mid_prices.index[j], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Ask_Close']]

#                 if trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
#                     nones.append(trade['start_date']) 

#                     trade = None
#                     break


#                 if trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
#                     trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
#                     day_fees = calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

#                     if trade_amount + day_fees > 0:
#                         buys.append(trade['start_date'])

#                     trade = None
#                     break

#                 if trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
#                     nones.append(trade['start_date'])

#                     trade = None
#                     break

#                 if trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
#                     trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
#                     day_fees = calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

#                     if trade_amount + day_fees > 0:
#                         sells.append(trade['start_date'])

#                     trade = None
#                     break
    
#     buys_list.append(buys)
#     sells_list.append(sells)
#     nones_list.append(nones)

#     print(f'Buys: {len(buys)}')
#     print(f'Sells: {len(sells)}')
#     print(f'Nones: {len(nones)}\n')





buys_list = []
sells_list = []
nones_list = []

value_per_pip = 1.0
amounts_per_day = [-0.00008, -0.0001, -0.00012]
spread_cutoff = 0.10
risk_reward_ratio = 1.5

def get_n_units(trade_type, stop_loss, ask_open, bid_open, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk = ask_open - stop_loss if trade_type == 'buy' else stop_loss - bid_open
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'Jpy' else pips_to_risk * 100

    if second == 'Usd':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'Jpy' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units

def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

for idx in range(len(currencies)):
    currency_pair, df, df_long, bid_ask_mid_prices = currencies[idx], dfs[idx], df_longs[idx], bid_ask_mid_prices_list[idx]
    rounding = 3 if 'Jpy' in currency_pair else 5
    buys, sells, nones = [], [], []
    trade, prev_year = None, None

    print(f'RUNNING SIMULATION FOR {currency_pair}...')

    for i in range(look_back_size, len(df)):
        curr_date = df.loc[df.index[i], 'Date']
        if prev_year is None or curr_date.year > prev_year:
            prev_year = curr_date.year
            print(prev_year)
            print(f'Buys: {len(buys)}')
            print(f'Sells: {len(sells)}')
            print(f'Nones: {len(nones)}\n')
        ema200_2, ema100_2, rsi2, macd2, macdsignal2 = df.loc[df.index[i - 2], ['ema200_x', 'ema100_x', 'rsi_x', 'macd_x', 'macdsignal_x']]
        ema200_1, ema100_1, rsi1, adx1, macd1, macdsignal1 = df.loc[df.index[i - 1], ['ema200_x', 'ema100_x', 'rsi_x', 'adx_x', 'macd_x', 'macdsignal_x']]
        mid_open2, mid_close2, mid_low2, mid_high2 = bid_ask_mid_prices.loc[bid_ask_mid_prices.index[i - 2], ['Mid_Open', 'Mid_Close', 'Mid_Low', 'Mid_High']]
        mid_open1, mid_close1, mid_low1, mid_high1 = bid_ask_mid_prices.loc[bid_ask_mid_prices.index[i - 1], ['Mid_Open', 'Mid_Close', 'Mid_Low', 'Mid_High']]
        curr_ao, curr_bo, curr_mid_open, curr_ask_low, curr_bid_high = bid_ask_mid_prices.loc[bid_ask_mid_prices.index[i], ['Ask_Open', 'Bid_Open', 'Mid_Open', 'Ask_Low', 'Bid_High']]
        spread = abs(curr_ao - curr_bo)

        macd_vals = [0, macd2, macdsignal2, macd1, macdsignal1]

        macd_buy_signal = macd2 < macdsignal2 and macd1 > macdsignal1 and max(macd_vals) == 0
        macd_sell_signal = macd2 > macdsignal2 and macd1 < macdsignal1 and min(macd_vals) == 0

        if trade is None:
            if macd_buy_signal:
                lowest_low = min(list(bid_ask_mid_prices.loc[bid_ask_mid_prices.index[i - 12:i], 'Mid_Low']))
                
                open_price = float(curr_ao)
                pullback = float(lowest_low) - spread

                stop_loss = round(pullback, rounding)

                if stop_loss < open_price:
                    curr_pips_to_risk = open_price - stop_loss

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), rounding)

                        n_units = get_n_units('buy', stop_loss, curr_ao, curr_bo, curr_mid_open, currency_pair)

                        trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                        'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                                        'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

            elif macd_sell_signal:
                highest_high = max(list(bid_ask_mid_prices.loc[bid_ask_mid_prices.index[i - 12:i], 'Mid_High']))

                open_price = float(curr_bo)
                pullback = float(highest_high) + spread

                stop_loss = round(pullback, rounding)

                if stop_loss > open_price:
                    curr_pips_to_risk = stop_loss - open_price

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), rounding)

                        n_units = get_n_units('sell', stop_loss, curr_ao, curr_bo, curr_mid_open, currency_pair)

                        trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

        if trade is not None:
            for j in range(i, len(df)):
                curr_date = df.loc[df.index[j], 'Date']
                curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = bid_ask_mid_prices.loc[bid_ask_mid_prices.index[j], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Ask_Close']]

                if trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
                    nones.append(trade['start_date']) 

                    trade = None
                    break


                if trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
                    trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
                    day_fees = calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

                    if trade_amount + day_fees > 0:
                        buys.append(trade['start_date'])

                    trade = None
                    break

                if trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
                    nones.append(trade['start_date'])

                    trade = None
                    break

                if trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
                    trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
                    day_fees = calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

                    if trade_amount + day_fees > 0:
                        sells.append(trade['start_date'])

                    trade = None
                    break
    
    buys_list.append(buys)
    sells_list.append(sells)
    nones_list.append(nones)

    print(f'Buys: {len(buys)}')
    print(f'Sells: {len(sells)}')
    print(f'Nones: {len(nones)}\n')

RUNNING SIMULATION FOR Eur_Gbp...
2018
Buys: 0
Sells: 0
Nones: 0

2019
Buys: 30
Sells: 37
Nones: 130

2020
Buys: 77
Sells: 87
Nones: 324

2021
Buys: 141
Sells: 131
Nones: 520

2022
Buys: 165
Sells: 148
Nones: 596

Buys: 199
Sells: 178
Nones: 704



In [14]:
buy_indices_list = []
sell_indices_list = []
nones_indices_list = []

for i in range(len(dfs)):
    buys, sells, nones, df = buys_list[i], sells_list[i], nones_list[i], dfs[i]
    
    buy_indices = [df.index[df['Date'] == curr_date] - 1 for curr_date in buys]
    sell_indices = [df.index[df['Date'] == curr_date] - 1 for curr_date in sells]
    nones_indices = [df.index[df['Date'] == curr_date] - 1 for curr_date in nones]

    buy_indices_list.append(buy_indices)
    sell_indices_list.append(sell_indices)
    nones_indices_list.append(nones_indices)

    print(len(buy_indices))
    print(len(sell_indices))
    print(len(nones_indices))

199
178
704


In [15]:
i = 5000
foo = dfs[0].iloc[i - look_back_size + 1:i + 1, 1:]
correct_shape = grab_image_data(foo).shape
correct_shape

(150, 23, 23)

In [16]:
def get_sequential_data():
    no_actions = []
    buys = []
    sells = []

    for z in range(len(dfs)):
        df, df_long = dfs[z], df_longs[z]

        buy_indices, sell_indices, nones_indices = buy_indices_list[z], sell_indices_list[z], nones_indices_list[z]

        for i in buy_indices:
            if len(i) == 1:
                i = i[0]
                seq = df.iloc[i - look_back_size + 1:i + 1, 1:]

                if seq.shape == correct_shape[:-1] and not seq.isnull().values.any():
                    seq = grab_image_data(seq)
                    buys.append([seq, np.array([0, 1, 0])])

        for i in sell_indices:
            if len(i) == 1:
                i = i[0]
                seq = df.iloc[i - look_back_size + 1:i + 1, 1:]

                if seq.shape == correct_shape[:-1] and not seq.isnull().values.any():
                    seq = grab_image_data(seq)
                    sells.append([seq, np.array([0, 0, 1])])

        for i in nones_indices:
            if len(i) == 1:
                i = i[0]
                seq = df.iloc[i - look_back_size + 1:i + 1, 1:]

                if seq.shape == correct_shape[:-1] and not seq.isnull().values.any():
                    seq = grab_image_data(seq)
                    no_actions.append([seq, np.array([1, 0, 0])])

    np.random.shuffle(no_actions)
    np.random.shuffle(buys)
    np.random.shuffle(sells)

    lower = min(len(no_actions), len(buys), len(sells))

    no_actions = no_actions[:int(lower * 1.0)]

    sequential_data = no_actions + buys + sells
    np.random.shuffle(sequential_data)

    return sequential_data

In [17]:
sequential_data = get_sequential_data()

In [18]:
len(sequential_data)

555

In [19]:
training_proportion = 0.70
train_test_cutoff_index = int(len(sequential_data) * training_proportion)

train_set = sequential_data[0:train_test_cutoff_index]
test_set = sequential_data[train_test_cutoff_index:]

print('Dataset shapes:')
print(len(train_set))
print(len(test_set))

Dataset shapes:
388
167


In [20]:
x_train = []
y_train = []

for seq, target in train_set:
  x_train.append(seq)
  y_train.append(target)

x_test = []
y_test = []

for seq, target in test_set:
  x_test.append(seq)
  y_test.append(target)

x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

In [21]:
x_train.shape

(388, 150, 23, 23)

In [22]:
y_train.shape

(388, 3)

In [23]:
n_actions = 3
input_data_shape = x_train.shape[1:]

model = Sequential()

model.add(Conv2D(filters = 16, kernel_size = (3,3), padding ='Same', activation ='relu', input_shape = input_data_shape))
model.add(Conv2D(filters = 32, kernel_size = (3,3), padding = 'Same', activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same', activation ='relu'))

model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
# model.add(Dense(128, activation = "relu"))
# model.add(Dropout(0.5))
# model.add(Dense(128, activation = "relu"))
# model.add(Dropout(0.5))
model.add(Dense(16, activation = "relu"))
# model.add(Dropout(0.25))
# model.add(Dense(32, activation = "relu"))
model.add(Dense(n_actions, activation = "softmax"))

In [24]:
# Hyperparameters
n_epochs = 100
batch_size = 32
n_steps = len(x_train) // batch_size 
mean_loss = tf.keras.metrics.Mean() 
optimizer = tf.keras.optimizers.Adam()
loss_fn = tf.keras.losses.categorical_crossentropy
metrics = [tf.keras.metrics.CategoricalAccuracy()]
# loss_fn = tf.keras.losses.MeanAbsoluteError
# metrics = [tf.keras.metrics.MeanAbsoluteError()]

In [25]:
early_stop = EarlyStopping(monitor='val_accuracy', verbose=1, patience=n_epochs)
model_checkpoint = ModelCheckpoint(f'/Users/mymac/forex_macd_cnn_{currencies[0]}', monitor='val_accuracy', save_best_only=True, verbose=1)

optimizer = Adam()

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [26]:
history = model.fit(
    x_train, y_train,
    batch_size=batch_size,
    epochs=n_epochs,
    validation_data=(x_test, y_test),
    callbacks=[early_stop, model_checkpoint]
)

Epoch 1/100
13/13 [==============================] - ETA: 0s - loss: 1.4459 - accuracy: 0.3479
Epoch 00001: val_accuracy improved from -inf to 0.31138, saving model to /Users/mymac/forex_macd_cnn_Eur_Gbp
INFO:tensorflow:Assets written to: /Users/mymac/forex_macd_cnn_Eur_Gbp/assets
13/13 [==============================] - 2s 119ms/step - loss: 1.4459 - accuracy: 0.3479 - val_loss: 1.0987 - val_accuracy: 0.3114
Epoch 2/100
11/13 [========================>.....] - ETA: 0s - loss: 1.0833 - accuracy: 0.4034
Epoch 00002: val_accuracy improved from 0.31138 to 0.42515, saving model to /Users/mymac/forex_macd_cnn_Eur_Gbp
INFO:tensorflow:Assets written to: /Users/mymac/forex_macd_cnn_Eur_Gbp/assets
13/13 [==============================] - 1s 67ms/step - loss: 1.0906 - accuracy: 0.3918 - val_loss: 1.0522 - val_accuracy: 0.4251
Epoch 3/100
11/13 [========================>.....] - ETA: 0s - loss: 1.0349 - accuracy: 0.4631
Epoch 00003: val_accuracy improved from 0.42515 to 0.56886, saving model to /